In [35]:
import time
import rasterio
import numpy as np
import math
from pathlib import Path 
from osgeo import gdal
import glob
import geopandas as gpd
import pandas as pd

# 0. Data Check
## treepoint data list
- manup_treepoint.shp
- mandown_treepoint.shp
- br_rest1_treepoint.shp
- br_rest2_treepoint.shp

In [83]:
tp_up = gpd.GeoDataFrame.from_file("tree_points/manup_treepoint.shp")
tp_down = gpd.GeoDataFrame.from_file("tree_points/mandown_treepoint.shp")
tp_br1 = gpd.GeoDataFrame.from_file("tree_points/br_rest1_treepoint.shp")
tp_br2 = gpd.GeoDataFrame.from_file("tree_points/br_rest2_treepoint.shp")
tp = gpd.GeoDataFrame( pd.concat( [tp_up, tp_down, tp_br1, tp_br2], ignore_index=True) )

In [84]:
# print(tp_up.info())
# print(tp_down.info())
# print(tp_br1.info())
# print(tp_br2.info())
# print(tp.info())

In [85]:
tp_up

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry
0,1401049,40.804813,-73.950177,Acer campestre - hedge maple,Rosidae,POINT (-73.95018 40.80481)
1,270934,40.809214,-73.951732,Zelkova serrata - Japanese zelkova,Hamamelididae,POINT (-73.95173 40.80921)
2,1401056,40.804494,-73.949422,Prunus serrulata 'Not Green leaf' - 'Not Green...,Rosidae,POINT (-73.94942 40.80449)
3,1401045,40.805001,-73.950625,Zelkova serrata - Japanese zelkova,Hamamelididae,POINT (-73.95062 40.80500)
4,1401047,40.804900,-73.950385,Quercus velutina - black oak,Hamamelididae,POINT (-73.95038 40.80490)
...,...,...,...,...,...,...
28268,5273853,40.793366,-73.934973,Platanus x acerifolia - London planetree,Hamamelididae,POINT (-73.93497 40.79337)
28269,5605087,40.814567,-73.924720,Quercus palustris - pin oak,Hamamelididae,POINT (-73.92472 40.81457)
28270,5650334,40.813503,-73.928576,Platanus x acerifolia - London planetree,Hamamelididae,POINT (-73.92858 40.81350)
28271,5605086,40.814642,-73.924666,Quercus palustris - pin oak,Hamamelididae,POINT (-73.92467 40.81464)


In [86]:
## subcalss string to number
from varname import nameof
for i in [tp_up, tp_down, tp_br1, tp_br2, tp]:
    # replace with *
    i['subclass'] = i['subclass'].fillna('*')
    
    # subclass string to int
    globals()[f'{i}_sc'] = i['subclass'].to_frame() # 열 하나만 추출하면 series로 type 바뀌어서 다시 dataframe으로 바꿔줘야 함.
    sc_list = {'subclass':{'Rosidae':'1', 'Hamamelididae':'2', 'Asteridae':'3',
                           'Dilleniidae':'4', 'Magnoliidae':'5', '*':'6'}}
    sc = globals()[f'{i}_sc'].replace(sc_list) # sc_list의 내용을 (tp_up_sc)에 대체해라.
    #print(f'{sc.shape} {nameof(i)}')
    
    # change
    globals()[f'{i}'] = sc
    print(sc)
    # change
    # 그냥 나머지 열 다 지우고 subclass만 살리면 concat 안 써도 됨. 근데 lat long.. 필요없겠지? geodataframe이면..
#     i.drop(['subclass', "OBJECTID", "Lat", "Long", "geometry"], axis = 1, inplace=True)
#     print(sc.head())
#     print()
#     globals()[f'{i}'] = pd.concat([i, sc], axis = 1) # 왜 concat이 안 될까...
    #print(i.info())

      subclass
0            1
1            2
2            1
3            2
4            2
...        ...
28268        2
28269        2
28270        2
28271        2
28272        2

[28273 rows x 1 columns]
      subclass
0            4
1            1
2            1
3            4
4            4
...        ...
24123        3
24124        2
24125        1
24126        2
24127        3

[24128 rows x 1 columns]
      subclass
0            2
1            3
2            2
3            1
4            1
...        ...
22233        1
22234        2
22235        6
22236        2
22237        2

[22238 rows x 1 columns]
      subclass
0            2
1            2
2            2
3            1
4            2
...        ...
11155        1
11156        6
11157        2
11158        1
11159        3

[11160 rows x 1 columns]
      subclass
0            1
1            2
2            1
3            2
4            2
...        ...
85794        1
85795        6
85796        2
85797        1
85798      

In [88]:
tp_up

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry
0,1401049,40.804813,-73.950177,Acer campestre - hedge maple,Rosidae,POINT (-73.95018 40.80481)
1,270934,40.809214,-73.951732,Zelkova serrata - Japanese zelkova,Hamamelididae,POINT (-73.95173 40.80921)
2,1401056,40.804494,-73.949422,Prunus serrulata 'Not Green leaf' - 'Not Green...,Rosidae,POINT (-73.94942 40.80449)
3,1401045,40.805001,-73.950625,Zelkova serrata - Japanese zelkova,Hamamelididae,POINT (-73.95062 40.80500)
4,1401047,40.804900,-73.950385,Quercus velutina - black oak,Hamamelididae,POINT (-73.95038 40.80490)
...,...,...,...,...,...,...
28268,5273853,40.793366,-73.934973,Platanus x acerifolia - London planetree,Hamamelididae,POINT (-73.93497 40.79337)
28269,5605087,40.814567,-73.924720,Quercus palustris - pin oak,Hamamelididae,POINT (-73.92472 40.81457)
28270,5650334,40.813503,-73.928576,Platanus x acerifolia - London planetree,Hamamelididae,POINT (-73.92858 40.81350)
28271,5605086,40.814642,-73.924666,Quercus palustris - pin oak,Hamamelididae,POINT (-73.92467 40.81464)


# 1. Point Sampling

In [41]:
# file에 영상 넣기 # NDVIup_file, NDVIdown_file, NDVIbr_file
for i in ['NDVI', 'SBI', 'GVI', 'YVI', 'WBI']:
    for j in ['up', 'down', 'br']:
        globals()[f'{i}{j}_file'] = '../KOMPSAT 위성영상/데이터 전처리/4. data/'+j+'/'+i+'.tif'
        globals()[f'{i}{j}_src'] = rasterio.open(globals()[f'{i}{j}_file'])
        

In [42]:
## 반복변수명과 값 확인
# for i in globals():
#     if i[-4:] == 'file':
#         print(f'{i}: {globals()[i]}')
#     elif i[-3:] == 'src':
#         print(f'{i}: {globals()[i]}')
#     else:
#         pass

In [49]:
# just NDVI point sampling
for i in [tp_up, tp_down, tp_br1, tp_br2, tp]:
    for j in ['up', 'down', 'br']:
        coord_list = [(x,y) for x,y in zip(i['geometry'].x , i['geometry'].y)] # 포인트의 좌표를 리스트로 추출
        #globals()[f'n{i}'] = i.drop(["OBJECTID", "Lat", "Long", "geometry"], axis=1)
        #ntp = globals()[f'n{i}']
        print(type(coord_list))

        i[Path('NDVI'+j+'_file').stem] = [x for x in 'NDVI'+j+'_src'.sample(coord_list)] # 포인트 샘플링
        i['NDVI'] = i['NDVI'].astype('float64')
tp_up.head()


area = ['up', 'down', 'br]
for i in [tp_up, tp_down, tp_br1, tp_br2, tp]:
    coord_list = [(x,y) for x,y in zip(i['geometry'].x , i['geometry'].y)] # 포인트의 좌표를 리스트로 추출
    #globals()[f'n{i}'] = i.drop(["OBJECTID", "Lat", "Long", "geometry"], axis=1)
    #ntp = globals()[f'n{i}']
    print(type(coord_list))

    i[Path('NDVI'+j+'_file').stem] = [x for x in 'NDVI'+j+'_src'.sample(coord_list)] # 포인트 샘플링
    i['NDVI'] = i['NDVI'].astype('float64')
tp_up.head()

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


,OBJECTID,Lat,Long,GenusSpeci,geometry
0,1401049,40.804813,-73.950177,Acer campestre - hedge maple,POINT (-73.95018 40.80481)
1,270934,40.809214,-73.951732,Zelkova serrata - Japanese zelkova,POINT (-73.95173 40.80921)
2,1401056,40.804494,-73.949422,Prunus serrulata 'Not Green leaf' - 'Not Green...,POINT (-73.94942 40.80449)
3,1401045,40.805001,-73.950625,Zelkova serrata - Japanese zelkova,POINT (-73.95062 40.80500)
4,1401047,40.804900,-73.950385,Quercus velutina - black oak,POINT (-73.95038 40.80490)


In [ ]:
NDVI_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/NDVI.tif'
NDVI_src = rasterio.open(NDVI_file) # 파일 읽기
gdf[Path(NDVI_file).stem] = [x for x in NDVI_src.sample(coord_list)] # 포인트 샘플링
gdf[Path(NDVI_file).stem] = gdf[Path(NDVI_file).stem].astype('float64')
gdf.head()

In [ ]:
# ndvi 0.2 이하 point 제거
# 1. 포인트샘플링, 2. 제거

In [61]:
# NDVI 0.2 이하 point 제거 이후, 3by3 생성
def pointsampling(area, file, src, coord_list, feature):
    for i in range(0, 9):
        WIDTH = 0.00000599299783809013749338974087784241
        HEIGHT = 0.0000059299783809007439469768700121737
        coord_list0= [(x,y) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list1= [(x-WIDTH, y+HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list2= [(x, y+HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list3= [(x+WIDTH,y+HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list4= [(x+WIDTH,y) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list5= [(x+WIDTH,y-HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list6= [(x,y-HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list7= [(x-WIDTH,y-HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list8= [(x-WIDTH,y) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        
        # 합치는 것만 생각해바
        tp02[Path(file).stem] = [x for x in src.sample(coord_list+'i')] # 포인트 샘플링
        tp02[feature] = tp02[feature].astype('float64') # 샘플링 값 dtype str to float64
        tp02[feature].drop(tp02[feature] == 0.0, inplace=True) # 그.. up, down, br을 합치고 싶은데... 어케할가,,
        tp02.rename(columns={feature: feautre+'_'+i}, inplace=True)
    tp02.head()
    print(len(tp02.index))
    return tp02

In [63]:
tpup = pointsampling(tp_up, NDVIup_file, NDVIdown_src, coord_list, "NDVI")

AttributeError: 'float' object has no attribute 'index'

In [ ]:
ndvi_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/manup2017/NDVI.tif'
ndvi_src = rasterio.open(ndvi_file) # 파일 읽기
tp[Path(ndvi_file).stem] = [x for x in ndvi_src.sample(coord_list)] # 포인트 샘플링
tp[Path(ndvi_file).stem] = tp[Path(ndvi_file).stem].astype('float64')

In [53]:
tp[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list)] # 포인트 샘플링
#tp[Path(NDVIup_file).stem] = tp[Path(NDVIup_file).stem].astype('float64')
tp[Path(NDVIup_file).stem]

0          [0.061114295761074494]
1          [-0.06895863107852732]
2        [-0.0018042394869449067]
3          [0.038802622168380596]
4           [0.22797190493318442]
                   ...           
85794                       [0.0]
85795                       [0.0]
85796                       [0.0]
85797                       [0.0]
85798                       [0.0]
Name: NDVI, Length: 85799, dtype: object

In [54]:
tp[Path(NDVIup_file).stem] = tp[Path(NDVIup_file).stem].astype('float64')
tp[Path(NDVIup_file).stem]

0        0.061114
1       -0.068959
2       -0.001804
3        0.038803
4        0.227972
           ...   
85794    0.000000
85795    0.000000
85796    0.000000
85797    0.000000
85798    0.000000
Name: NDVI, Length: 85799, dtype: float64

In [55]:
type(tp[Path(NDVIup_file).stem])

pandas.core.series.Series